Orca is a class used to run orca(In the forum(https://orcaforum.kofo.mpg.de/index.php) you can find the manual and software download address) in python enviroment and finish some quantum chemistry calculaitons with settings of dicts. And the class Orca_workflow realizes a workflow protocal based on the Orca class, which can conduct a calculation task of multi-steps with a predefined settings of a dict. This will bring much convenience when lots of tasks of same kind are to be finished in works.

# Orca class test

In [2]:
from calculator.orca.orca import Orca
from calculator.orca.orca_workflow import Orca_workflow
import  os,re,sys,json
import numpy as np

Before calculation, orca and openmpi should be installed. You can refer to the install script(https://github.com/dcccc/calculator/blob/main/example/orca/how_to_install_orca.sh) or corresponding content in orca manual. After installation, executable file paths of orca and openmpi orca and openmpi can be obtianed. They are setted with a dict. 

In [3]:
#           executable file path of orca 
orca_exe = {"orca":"/mnt/c/TDDownload/orca_5_0_4_linux_x86-64_shared_openmpi411/orca",
#           executable file path of openmpi. when the openmpi is not presented, the orca can only
#           be ran in a serial mode(one core is used in the calculaiton)
            "openmpi":"/home/flw/software/openmpi/openmpi411/bin/mpirun",}


In [6]:
# create a instance of Orca class with setting of orca and openmpi dict setting of executable file path 
orca = Orca(orca_exe=orca_exe)

The parameters for the calculation are also dicts, including "para" for the content of calculaiton method and "mol" for the molecule system to be studied.

The detail of "para" parameters is :

In [ ]:
para = {
#  the memory to be used for every core, corresponding to the "maxcore" keyword in orca. No default value.
#  and won't be written in input file when not setted
# value type : int
"maxcore": None,

# the number of process used in calculaiton, corresponding to the "nprocs" keyword in orca. No default 
# value. And "nprocs" keyword line won't be written in input file when not presented, so the orca will 
# be run in a serial mode. Of cuause, you can use "pal" keyword in keyword line for parallel computing
# while not using this key item.
# value type : int
"nprocs" : 1,

# the keyword line in orca input file, corresponding to the "nprocs" keyword in orca,
#  which can not be omitted. When not presented, a error will be raised
# value type : string
"keyword":""

# the detail control parameters for calculation. Every item in the the dict of "detail_para" is a 
# "input block". The name of "input block" item is the block name of "input block", and the item 
#  names and values of input block" items are the setting and values for the "input block".No default 
#  value, and won't be written in input file when not setted 
# value type : dict
"detail_para":{}
    
# same to the previous "detail_para", and it is for those detail control parameters blocks which has 
# to be placed after molecule coordiante part, like "eprnmr"  block.No default value, and won't be 
# written in input file when not setted 
# value type : dict
"para_after_coord"{}
}



A sample "para" is like 

In [11]:
para = {"maxcore": 2000, 
        "nprocs": 4 ,
        "keyword":" b3lyp/g d3bj  DEF2-SVP ",
        "detail_para":{"ELPROP":{"POLAR":1}, 
                       "CPCM": {"smd":"true", "SMDSOLVENT" :' "1-OCTANOL" '}},
        "para_after_coord":{"EPRNMR":{"NUCLEI":"= ALL H {SHIFT, SSALL}"}} }
        

And the detail of "mol" parameters is :

In [ ]:
mol  = {
# the charge of molecule, the defualt value is 0
# value type : float
"charge":0, 
    
# the spin of molecule, the defualt value is 1    
# value type : int
"spin":1, 

# the atom types and coordinates of molecule, which can not be omitted.
# value type : string 
"coord":"", 

# the type of coordinate of molecule, value can be "xyz" for xyz type or "int" for internal coordinate
# value type : string
"coord_type":""}

A sample mol is like

In [9]:
mol_xyz = '''
O         -3.56626        1.77639        0.00000
H         -2.59626        1.77639        0.00000
H         -3.88959        1.36040       -0.81444
'''    
    
mol  = {"charge":0, "spin":1, "coord":mol_xyz, "coord_type":"xyz"}

When para and mol are prepared, the orca input file content can be printed for check

In [12]:
orca.write_input(mol, para,  print_input=True, write_input=False)

%MaxCore  2000 
%PAL NPROCS 4 END 
!  b3lyp/g d3bj  DEF2-SVP 
%ELPROP
    POLAR      1  
END
%CPCM
    smd        true  
    SMDSOLVENT  "1-OCTANOL"   
END
* xyz  0   1 
 
O         -3.56626        1.77639        0.00000
H         -2.59626        1.77639        0.00000
H         -3.88959        1.36040       -0.81444
*
%EPRNMR
    NUCLEI     = ALL H {SHIFT, SSALL}  
END



When the para and mol are checked without error, the the calculation can be ran

In [6]:
# mpirun_para can be setted, which will be added after "mpirun "  command, and the " --oversubscribe" is 
# added by default. The orca jobname is "orca".
# value type : string
os.mkdir("./orca_test")
os.chdir("./orca_test")
orca.write_input(mol, para)
orca.run(mpirun_para="") 

After finish calculaiton, some kind of results can be read with function method in Orca class.The detail of those functions can be referred to from the source code of functions.

In [ ]:

Orca.get_dipole()     # get dipole  
Orca.get_ecd()        # get ecd absorption spectra 
Orca.get_energy(energy_list=[])     # get energy item in "orca_perpoty.txt" file, energy_list is a list of 
                                    # strings, the member in list is the energy item to be extracted from 
                                    # "orca_perpoty.txt". When not founded, None is returned
Orca.get_hessian()    # get hessian  
Orca.get_ir()         # get ir data   
Orca.get_mo_energy()  # get molecule orbit energy   
Orca.get_nmr()        # get nmr result   
Orca.get_raman()      # get raman result   
Orca.get_runtime()    # get the computation time and coretime of current task
Orca.get_soc()        # get the spin-orbit coupling result
Orca.get_trajectory() # get the trajectory result
Orca.get_uv()         # get the UV/Vis  absorption spectra result

In [9]:
# get "Total Energy" in "orca_perpoty.txt" 
Orca.get_energy(energy_list=["Total Energy"])

{'energy': 76.368106346207, 'Total Energy': -76.368106}

In [8]:
# get nmr result
Orca.get_nmr()

array([[ 1.    ,  2.    , -2.3283]])

# workflow test

When orca and openmpi are installed, a dict of setting can be defined to conduct a claculation workflow.The parameters for the calculation is a dict, which includes "task_step" for the task step name of calculaiton steps and the calculaiton parameters named with the task step name of calculaiton steps.

The detail of "workflow_dict" parameters is :

In [4]:
workflow_dict ={
    # The name list of task steps, and shouldn't be []. Every task step calculation will be ran or skipped 
    # if the task step has a valid corresponding settings in the workflow_dict.
    # the workflow_dict will be saved as "workflow.json" file after every task step finished .
    "task_step":[], 
    # The parameter and molecule system settings for the task steps in the task step list.
    "task_step_name" : {
        # The molecule system setting, which is same to the "mol" in previos orca calculation setting. 
        # When it is not setted, the calcuation result molecule of last task step will be used. 
        "mol" : {},
        # the calculation setting for the task step. It can not be omitted.
        "para" :{},
        # the run mode of this task step. 
        # The default value is None, which means a directory with name of this task step iwill be 
        # created, and calculation will be ran in it. If there is a directory of name of this task 
        # step created before, then the directory will be backuped to a name of "***_bak" and with 
        # a number after it. 
        
        # "continue_run" : enter into the previous directory with name of this task step created before, 
        # and run the calculation. This mode is for the restart calculation when the previous calculation
        # of this task step is stopped unexpectly.    
        
        # "cmd" :  the shell command script lines in "cmd" item are to be ran at current directory. 
        # The "para" and "mol" are not used in this mode
        
        #"skipped" : this task step will be skipped 
        "run_type":None,
        
        # the shell command script line(s), which are to be ran at current directory. It is a string
        # or list of strings.
        "cmd" : [],
        
        # the result items of to be read from result files after orca calculation of task step.
        # the "xyz" , "energy" and "runtime" are default. "xyz" is the molecule structure result, 
        # "energy" is the last "FINAL SINGLE POINT ENERGY" in "orca.out" file, and "runtime"  is the 
        # time taken to finish the calculation of the task step
        # other valid items are "dipole", "ecd", "energy", "hessian ", "ir", "mo_energy ", "nmr", 
        # "raman", "runtime", "soc", "trajectory", "uv", which corresponds the results read function 
        # method in Orca class
        # the value can be a dict or a list. When the value is a dict, the value of item "energy" can be 
        # a list of strings, then the corresponding energy in "energy" item list will be read from the 
        # "orca_property.txt" file.        
        "result" :[] ,
        
        # the files are to be deleted after orca calculation of this task step
        "delete_files":[]
    }
}

A sample of workflow dict is as below.

NOTICE: The parameters and molecule settings are not for production calculaiton,they are only for test and presention of workflow. 

In [24]:
mol_xyz = '''
O         -3.56626        1.77639        0.00000
H         -2.59626        1.77639        0.00000
H         -3.88959        1.36040       -0.81444
'''    
    
mol  = {"charge":0, "spin":1, "coord":mol_xyz, "coord_type":"xyz"}

# optimize mol with b97-3c method
para_b973c = {"maxcore": 2000, 
              "nprocs": 4 ,
              "keyword":" opt b97-3c "}

# optimize mol which has been optimized by previous step b97-3c method with b3lyp fucntional, 
# DEF2-TZVP basis set at the solvent of water enviroment,  
para_b3lyp = {"maxcore": 2000, 
              "nprocs": 4 ,
              "keyword":" opt b3lyp/g d3bj  DEF2-TZVP RIJCOSX def2/J ",
              "detail_para":{"CPCM": {"smd":"true", "SMDSOLVENT" :' "WATER" '}}
             }

# calculate the single point energy, frequency, with 
# DEF2-TZVP basis set at the solvent of water enviroment. and get the result 
para_wb97mv = {"maxcore": 2000, 
              "nprocs": 4 ,
              "keyword":" wb97m-v  def2-TZVPP  RIJCOSX def2/J numfreq ",
              "detail_para":{"CPCM": {"smd":"true", "SMDSOLVENT" :' "WATER" '}}
                             
              }

# calculate the uv/vis absorption spectrum with double hybrid functional  
para_dh     = {"maxcore": 2000, 
              "nprocs": 1 ,
              "keyword":" ri-SCS-PBE-QIDH  def2-TZVPP  RIJCOSX def2/J def2-TZVPP/C ",
              "detail_para":{"TDDFT":{"nroots":5}, 
                             "mp2": {"Density": "relaxed"},
                             "CPCM": {"smd":"true", "SMDSOLVENT" :' "WATER" '}},
               }

workflow_dict = {"task_step":["1.opt_b973c", "2.opt_b3lyp", "3.sp_wb97mv", "4.tddft_SCS-PBE-QIDH"], 
                 
                 "1.opt_b973c": {"mol":mol, "para":para_b973c, "delete_files":["*gbw", "*densities"]},
                 
                 # the "mol" is not setted, so the result mol structure of step "1.opt_b973c" will be used
                 "2.opt_b3lyp": {"para":para_b3lyp, 
                                 "delete_files":["*gbw", "*densities", "*smd*", "*opt", "*cpcm"]},
                 
                 # the "mol" is not setted, so the result mol structure of step "2.opt_b3lyp" will be used
                 "3.sp_wb97mv": {"para":para_wb97mv,
                                 "result":["raman", "hessian", "ir"],
                                 "delete_files":["*gbw", "*densities", "*smd*", "*opt", "*cpcm"]},
                 
                 # the "mol" is not setted, so the input mol structure of step "3.sp_wb97mv" will be used
                 "4.tddft_SCS-PBE-QIDH": 
                                 # as the "run_type" is "skipped", so this task step will be skipped
                                {"run_type": "skipped",
                                 "para":para_dh, 
                                 "result":["uv"],
                                 "delete_files":["*gbw", "*densities", "*smd*", "*opt", "*cpcm", "*cis"]},
                 
                 # the step name of this task step is not in the "task_step" list, so this step will not 
                 # be ran
                 "5.cmd":{"cmd":["rm */*opt"]}
                 }

In [27]:
os.mkdir("./orca_workflow_test")
os.chdir("./orca_workflow_test")
workflow = Orca_workflow(orca_exe=orca_exe, workflow_json=workflow_dict)
workflow.task_run()